In [1]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

In [2]:
geo = SplineGeometry()
m = 0.6
geo.AddCircle(c=(0, 0), r=2, bc="cyl", maxh=m) # 0.04
mesh = Mesh(geo.GenerateMesh(maxh=m))
mesh2 = Mesh(geo.GenerateMesh(maxh=m))
mesh.Curve(3);
mesh2.Curve(3)
scene1 = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [3]:
def calcBc():
    r = 1/Integrate(1,mesh)*Integrate(x,mesh)
    return r
calcBc()

-5.080028269950915e-17

In [4]:
# vector space for shape gradient
VEC = H1(mesh, order=1, dim=2)
# Test and trial functions
PHI, X = VEC.TnT()
gfX = GridFunction(VEC)

In [5]:
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

dJOmega  = LinearForm(VEC)

# volume stuff
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)
vol = Parameter(1)
vol.Set(Integrate(surf_t,mesh))
vol_inv = Parameter(1)
vol_inv.Set(1/Integrate(surf_t,mesh))
# bc prep
bc_tx = CoefficientFunction(x)
bc_0x = 1/surf_0*Integrate(bc_tx,mesh)

# bc constraints
beta0 = 1e-1
beta = Parameter(beta0)
bc_x = Parameter(1)
bc_int = Parameter(1)
bc_int.Set(Integrate(bc_tx,mesh))
bc_x.Set(calcBc())

dJOmega += ((beta*((bc_x-bc_0x)**2)*dx)).DiffShape(X)

In [6]:
dJOmega.Assemble()
print("curvol: ", vol.Get())
print("Initial bc: ", bc_x.Get())
print(dJOmega.vec[0:5])

curvol:  12.566430839284152
Initial bc:  -5.080028269950915e-17
       0
      0

       0
      0

       0
      0

       0
      0

       0
      0




In [7]:
mesh.UnsetDeformation()
test = CoefficientFunction((0.8,1.2))
gfX.Set(test)
mesh.SetDeformation(gfX)
scene = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [8]:
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))
dJOmega.Assemble()
print("curvol: ", Integrate(1,mesh))
print(bc_x.Get())
print(abs(bc_0x-bc_x.Get()))
#print(dJOmega.vec)

curvol:  12.56643083928415
0.7999999999999996
0.7999999999999996


## try to do this as shape optimization
Barycenter:
$\Large bc = \frac{1}{\int_\Omega 1 \mathrm{dx}}\cdot\int_\Omega x \mathrm{dx}$ <br>
#### above as shape derivative:
$\Large-\frac{1}{|\Omega |^2}\cdot\int div(x)\mathrm{dx}\cdot(\int x \mathrm{dx})+$<br>
$\Large\frac{1}{|\Omega |}\cdot\int div(v)x_i \mathrm{dx}\cdot\int v_i \mathrm{dx}$ <br>
 unimplementierbar?!

In [9]:
# don't use 2 deformations, just set bc_0x to something else, let's see if we get there:
mesh.UnsetDeformation()
bc_0x = 1.5

gfset = GridFunction(VEC)

dJOmega2 = LinearForm(VEC)
# initial try in auto (wrong)
#dJOmega2 += ((beta*((bc_x-bc_0x)**2)*dx)).DiffShape(X)
# derivative by sturm (wrong)
#dJOmega2 += ((2*beta*((1/bc_x-bc_0x))*(1/bc_x)*dx)).DiffShape(X)
# another try (wrong)
# dJOmega2 += (2*(1/(vol_inv*bc_int)-bc_0x)*(1/(vol_inv*bc_int))*dx).DiffShape(X)
# new try
dJOmega2 += 2*beta*(bc_x-bc_0x)*((1/vol**2)*div(X)*x+(1/vol)*div(X)*x*sum(gfset.vecs[0].data)[0])*dx

gfset = GridFunction(VEC)

b = BilinearForm(VEC)
b += (InnerProduct(grad(X),grad(PHI))).Compile() *dx + (InnerProduct(X,PHI)).Compile()*dx

def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega2.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [10]:
bc_x.Set((1/surf_0)*Integrate(bc_tx,mesh))
bc_x.Get()

-5.080028269950915e-17

In [11]:
import time
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
iter_max = 100

for i in range(0,iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    
    b.Assemble()
    
    vol_inv.Set(1/Integrate(surf_t,mesh))
    vol.Set(Integrate(surf_t,mesh))
    bc_x.Set((1/vol.Get())*Integrate(bc_tx,mesh)) # needed for multiple iterations (already done once before)
    bc_int.Set(Integrate(bc_tx,mesh))
    #print(1/Integrate(1,mesh)*Integrate(x,mesh))
    print(bc_x.Get())
    dJOmega2.Assemble()
    SolveDeformationEquation()
    
    mesh.UnsetDeformation()
    
    gfxnorm = Norm(gfX.vec)
    scale = 0.1 / gfxnorm
    gfset.vec.data -= scale * gfX.vec
    
    time.sleep(0.1)

-5.080028269950915e-17
0.027932077808441386
0.05585240900613169
0.08374819370381457
0.11160677482548435
0.13941573952627767
0.16716301491991595
0.1948369556881842
0.222426421552814
0.24992084308403373
0.27731027486086873
0.30458543554618583
0.3317377349579224
0.3587592886769194
0.38564292110992676
0.41238215821171603
0.4389712112596988
0.4654049531726351
0.49167888888211253
0.5177891212154769
0.5437323136473814
0.5695056511397629
0.5951068001313029
0.6205338685697541
0.6457853667138714
0.67086016927368
0.6957574793134634
0.7204767942144156
0.7450178738845704
0.7693807113126743
0.7935655054891231
0.8175726366595419
0.8414026438332011
0.8650562044373006
0.8885341159871786
0.9118372796299373
0.9349666854130353
0.9579233991286591
0.9807085505877671
1.0033233231836576
1.0257689446126972
1.048046678628887
1.0701578177186128
1.0921036765917924
1.1138855863954287
1.1355048895650457
1.15696293523841
1.1782610751643792
1.1994006600474376
1.2203830362755865
1.2412095429856804
1.261881509426098
1.

In [12]:
bc_x.Get()

1.4998614940847232